<a href="https://colab.research.google.com/github/8persy/algoritms_colab/blob/main/%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%B2%D0%B5%D1%82%D0%B2%D0%B5%D0%B9_%D0%B8_%D0%B3%D1%80%D0%B0%D0%BD%D0%B8%D1%86_11_311_11_313.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Метод ветвей и границ

Данный метод применяется в задачах, когда при переборе значений мы делаем гипотезу и вычисляем решение и его оценку в двух случаях:
- В случае если гипотеза верна
- В случае если гипотеза неверна

Для каждого из новых решений проделываем ту же операцию и в конечном счете получим решение с лучшей оценкой


Полный разбор задачки тут

https://galyautdinov.ru/post/zadacha-kommivoyazhera

Рассмотрим этот метод на примере задачи коммивояжера

Смысл задачи:
- У нас есть N городов, связанным между друг другом
- Есть матрица с расстояниями м-ду городами
- Необходимо посетить каждый город хотя бы один раз и затем вернуться в исходный город
- Необходимо найти самый короткий путь

In [ ]:
import numpy as np

У нас есть 5 городов с вот такой матрицей расстояний

<img src="https://drive.google.com/uc?export=view&id=1bfNvGtXZ2OHA8XKow9XIpZJAeMbqQywo" height="200"/>

In [ ]:
routes = np.array([
    [np.inf, 20,     18,     12,     8],
    [5,      np.inf, 14,     7,      11],
    [12,     18,     np.inf, 6,     11],
    [11,     17,     11,     np.inf, 12],
    [5,      5,      5,      5,      np.inf],
])

Теперь попытаемся найти оценку самого минимального пути м-ду городами

Найдем эту оценку так:
- Возьмем минимумы по строкам (все самые короткие пути), просуммируем их
- Вычтем эти минимумы из ячеек соответствующих строчек
- Сделаем аналогичные шаги по столбцам

Получившиеся число будет минимально возможной оценкой

В чем суть?

Посредством такой операции мы на каждом шаге будем вычитать минимумы чтобы найти самые дешевые пути между городами

In [ ]:
mins = routes.min(axis=1)

m = np.repeat(mins, 5).reshape(5, 5)

r_routes = routes - m

r_routes

array([[inf, 12., 10.,  4.,  0.],
       [ 0., inf,  9.,  2.,  6.],
       [ 6., 12., inf,  0.,  5.],
       [ 0.,  6.,  0., inf,  1.],
       [ 0.,  0.,  0.,  0., inf]])

In [ ]:
m

array([[ 8.,  8.,  8.,  8.,  8.],
       [ 5.,  5.,  5.,  5.,  5.],
       [ 6.,  6.,  6.,  6.,  6.],
       [11., 11., 11., 11., 11.],
       [ 5.,  5.,  5.,  5.,  5.]])

In [ ]:
record = mins.sum()

record

35.0

In [ ]:
mins = r_routes.min(axis=0)

m = np.repeat(mins, 5).reshape(5, 5).T

c_routes = r_routes - m

c_routes

array([[inf, 12., 10.,  4.,  0.],
       [ 0., inf,  9.,  2.,  6.],
       [ 6., 12., inf,  0.,  5.],
       [ 0.,  6.,  0., inf,  1.],
       [ 0.,  0.,  0.,  0., inf]])

In [ ]:
record += mins.sum()

record

35.0

Таким образом, меньше чем за 35 объехать все города невозможно

Соберем все в одну функцию

In [ ]:
def remove_mins(arr: np.array):
  routes = np.array(arr)
  size = arr.shape[0]
  const_sum = 0

  row_mins = routes.min(axis=1)
  row_mins[row_mins == np.inf] = 0
  row_sum = row_mins.sum()
  const_sum += row_sum

  m = np.repeat(row_mins, size).reshape(size, size)
  routes = routes - m

  col_mins = routes.min(axis=0)
  col_mins[col_mins == np.inf] = 0
  col_sum = col_mins.sum()
  const_sum += col_sum

  m = np.repeat(col_mins, size).reshape(size, size).T
  routes = routes - m

  return routes, const_sum

In [ ]:
min_routes, record = remove_mins(routes)

min_routes

array([[inf, 12., 10.,  4.,  0.],
       [ 0., inf,  9.,  2.,  6.],
       [ 6., 12., inf,  0.,  5.],
       [ 0.,  6.,  0., inf,  1.],
       [ 0.,  0.,  0.,  0., inf]])

После всех махинаций получаем вот такую матрицу

Нули - это самые дешевые маршруты м-ду городами

Но какой ноль выбрать?

Хотелось бы выбрать такой ноль, чтобы посетить город, уехать потом из сложнее всего (ну туда все равно ехать, так хоть по самому короткому пути поедем)

Как такой ноль найти? Каждый ноль будем оценивать суммой минимумов по столбцам и строкам


<img src="https://drive.google.com/uc?export=view&id=1gpocaPmdNg0SSVfgF8L4q9bwk58UXK-H" height="200"/>

Тогда увидим что B - самый затратный город сейчас, туда и поедем

Выбираем ноль с наибольшей оценкой

<img src="https://drive.google.com/uc?export=view&id=1TfHvvA91CNQY5g203ZJRIUxb--bOzp5t" height="200"/>

Сделаем функцию поиска нуля с наибольшей оценкой

In [ ]:
def find_idxs(arr: np.array):
  maxmin = -1
  max_i = 0
  max_j = 0

  for i in range(5):
    for j in range(5):
      if arr[i][j] == 0:
        replaced = np.array(arr)
        replaced[i][j] = np.inf
        curr_min = replaced[:, j].min() + replaced[i].min()

        if curr_min > maxmin:
            maxmin = curr_min
            max_i = i
            max_j = j

  return max_i, max_j

In [ ]:
max_i, max_j = find_idxs(min_routes)

max_i, max_j

(4, 1)

Теперь у нас появилась гипотеза:
- В оптимальном маршруте есть путь м-ду E и B

Надо рассмотреть два варианта:
- E-B существует в лучшем маршруте
- E-B не существует в лучшем маршруте

Здесь и появляются бинарные деревья:
- Узел - это некоторое частичное решение задачи
- Его листья - возможные исходы
- Корень дерева - решение с первоначальной оценкой (35)

<img src="https://drive.google.com/uc?export=view&id=1EagoHeB9n-N8WJ9t9ciWbC4ta05znrD_" height="200"/>

Рассмотрим что делать, если E-B есть

Тогда нужно чтобы соблюдалось 3 условия:
- В B больше ниоткуда нельзя заезжать, кроме как из E
- В E больше никуда нельзя выезжать, кроме как в B
- Напрямую из B в E нельзя ехать

Т.е:
- стоимость проезда в строке E теперь равна бесконечности
- стоимость проезда в столюце B теперь равна бесконечности
- стоимость проезда из B в E теперь равна бесконечности

И дальше в решении для этой гипотезы будем рассматривать матрицу в которой как бы вычеркнули эти столбцы и строки, информации в них все равно полезной больше нет

<img src="https://drive.google.com/uc?export=view&id=1NcLfKLKjmnEFyu5cWxi053jT9mDR4Lwx" height="200"/>

<img src="https://drive.google.com/uc?export=view&id=1pi67ECT80uO1pLZUmqtnDycRlU_W8vXx" height="200"/>

Сделаем функцию преобразования матрицы если считаем что гипотеза верна

In [ ]:
def make_left_solution(i: int, j: int, arr: np.array):
  solution = np.array(arr)

  solution[i] = np.inf

  solution[:, j] = np.inf

  solution[j][i] = np.inf

  return solution

In [ ]:
left_solution = make_left_solution(max_i, max_j, min_routes)

left_solution

array([[inf, inf, 10.,  4.,  0.],
       [ 0., inf,  9.,  2., inf],
       [ 6., inf, inf,  0.,  5.],
       [ 0., inf,  0., inf,  1.],
       [inf, inf, inf, inf, inf]])

In [ ]:
min_left, left_record = remove_mins(left_solution)

l_record = record + left_record

l_record

35.0

А как быть если гипотезу не принимаем?

Тогда просто стоимость этого маршрута равна бесконечности

In [ ]:
right_solution = np.array(min_routes)

right_solution[max_i][max_j] = np.inf

right_solution

array([[inf, 12., 10.,  4.,  0.],
       [ 0., inf,  9.,  2.,  6.],
       [ 6., 12., inf,  0.,  5.],
       [ 0.,  6.,  0., inf,  1.],
       [ 0., inf,  0.,  0., inf]])

In [ ]:
min_right, right_record = remove_mins(right_solution)

r_record = record + right_record

r_record

41.0

Получим вот такое дерево

<img src="https://drive.google.com/uc?export=view&id=1lx442oXInJUi-e1i0DoTlqtHqxCFskWA" height="400"/>

Что теперь будем делать?

- Перебираем листы дерева
- Выбираем лист с наименьшей оценкой
- Для этого листа выдвигаем гипотезу и делаем 2 новых решения
- Новые решения становятся листами дерева

Перебираем листы до тех пор, пока не найдется лист нужной высоты

Будем хранить не все дерево, а только листы

Лист опишем такой структурой

In [ ]:
from dataclasses import dataclass

@dataclass
class Solution:
  arr: np.array
  path: list[list[int]]
  record: int

In [ ]:
def main():
    solution_pool: list[Solution] = []

    # Найти лучший лист
    def find_best_index():
        min_rec = 10_000
        idx = 0
        for i in range(len(solution_pool)):
            solution = solution_pool[i]
            if solution.record < min_rec:
                min_rec = solution.record
                idx = i

        return idx

    # Есть ли лист нужной высоты
    def should_continue():
        for solution in solution_pool:
            if len(solution.path) == routes.shape[0]:
                return False
        return True

    # Найти ответ после окончания перебора вариантов
    def find_answer():
        for solution in solution_pool:
            if len(solution.path) == routes.shape[0]:
                return solution
        return None

    # Создаем начальное решение
    min_routes, record = remove_mins(routes)

    # Записываем начальное решение в массив с изначальной оценкой
    solution_pool.append(
        Solution(
            arr=min_routes,
            record=record,
            path=[]
        )
    )

    while should_continue():
        # Ищем лучшее решение
        best_idx = find_best_index()

        best_solution = solution_pool[best_idx]

        # Убираем решение из массива т.к оно больше не лист
        solution_pool.pop(best_idx)

        arr = best_solution.arr

        # Находим лучший нолик, создаем гипотезу
        sol_i, sol_j = find_idxs(arr)

        # Генерим решение если гипотеза верна
        left_arr = make_left_solution(sol_i, sol_j, arr)
        min_left_arr, min_sum = remove_mins(left_arr)

        # Помещаем решение в массив
        solution_pool.append(
            Solution(
                arr=min_left_arr,
                record=best_solution.record + min_sum,
                path=[*best_solution.path, [sol_i, sol_j]]
            )
        )


        # Генерим решение если гипотеза не верна
        right_arr = np.array(arr)
        right_arr[sol_i][sol_j] = np.inf
        min_right_arr, min_sum = remove_mins(right_arr)

        # Помещаем решение в массив
        solution_pool.append(
            Solution(
                arr=min_right_arr,
                record=best_solution.record + min_sum,
                path=best_solution.path
            )
        )

    answer = find_answer()

    print(answer.path)

In [ ]:
main()

[[3, 1], [0, 4], [1, 0], [2, 3], [4, 2]]
